# Notebook 4: Dry bf bubble

The dry df bubble example. This considers the idea of recovery. This is where we wish to solve for a higher order field around the object of interest, which in this case, is a dry bubble. 

We begin by importing the required libraries and functions from Firedrake and Gusto:

In [4]:
from gusto import *
from firedrake import (IntervalMesh, ExtrudedMesh,
                       SpatialCoordinate, conditional, cos, pi, sqrt,
                       TestFunction, dx, TrialFunction, Constant, Function,
                       LinearVariationalProblem, LinearVariationalSolver,
                       FunctionSpace, BrokenElement, VectorFunctionSpace)
from firedrake.slope_limiter.vertex_based_limiter import VertexBasedLimiter
import sys

We now define the time-step size and simulation time.

In [5]:
dt = 1.0
deltax = 100.
tmax = 1000.

Make the mesh. This is an interval mesh that we extrude in the vertical direction.

In [6]:
L = 10000.
H = 10000.
nlayers = int(H/deltax)
ncolumns = int(L/deltax)
m = IntervalMesh(ncolumns, L)
mesh = ExtrudedMesh(m, layers=nlayers, layer_height=H/nlayers)

Define the lowest order degree in this recovered case.

In [7]:
# options
diffusion = False
degree = 0

Define the output. We will record the full velocity fields, as well as the perturbations to the potential temperature.

In [8]:
dirname = 'dry_bf_bubble_recovered'

output = OutputParameters(dirname=dirname,
                          dumpfreq=int(tmax / (5*dt)),
                          dumplist=['u'],
                          perturbation_fields=['theta'],
                          log_level='INFO')

Set up the state, using parameters corresponding to a compressible set-up. 

In [9]:
params = CompressibleParameters()

state = State(mesh,
              dt=dt,
              output=output,
              parameters=params)


gusto:INFO Physical parameters that take non-default values:
gusto:INFO 


Define the equations to solve, which are the compressible Euler equations. We neglect any diffusion options for this simulation.

In [11]:
diffusion_options = None

u_transport_option = "vector_advection_form" 

eqns = CompressibleEulerEquations(state, "CG", degree,
                                  u_transport_option=u_transport_option,
                                  diffusion_options=diffusion_options,
                                  no_normal_flow_bc_ids=[1, 2])


Define the initial conditions and the function spaces.

In [12]:
# Initial conditions
u0 = state.fields("u")
rho0 = state.fields("rho")
theta0 = state.fields("theta")

# spaces
Vu = state.spaces("HDiv")
Vt = state.spaces("theta")
Vr = state.spaces("DG")
x, z = SpatialCoordinate(mesh)

Initialise the state to be in hydrostatic balance

In [13]:
# Define constant theta_e and water_t
Tsurf = 300.0
theta_b = Function(Vt).interpolate(Constant(Tsurf))

# Calculate hydrostatic fields
compressible_hydrostatic_balance(state, theta_b, rho0, solve_for_rho=True)

More set up for the initial conditions

In [14]:
# make mean fields
rho_b = Function(Vr).assign(rho0)

# define perturbation
xc = L / 2
zc = 2000.
rc = 2000.
Tdash = 2.0
r = sqrt((x - xc) ** 2 + (z - zc) ** 2)
theta_pert = Function(Vt).interpolate(conditional(r > rc,
                                                  0.0,
                                                  Tdash * (cos(pi * r / (2.0 * rc))) ** 2))

# define initial theta
theta0.assign(theta_b * (theta_pert / 300.0 + 1.0))


Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.ExtrudedMeshTopology object at 0x7f8c62e5e3a0>, TensorProductElement(FiniteElement('Discontinuous Lagrange', interval, 0, variant='equispaced'), FiniteElement('Lagrange', interval, 1, variant='equispaced'), cell=TensorProductCell(interval, interval)), name='Vtheta', index=2, component=None), Mesh(VectorElement(TensorProductElement(FiniteElement('Lagrange', interval, 1), FiniteElement('Lagrange', interval, 1), cell=TensorProductCell(interval, interval)), dim=2), 4)), 14)

More set up and pass this information to the state instance.

In [15]:
# find perturbed rho
gamma = TestFunction(Vr)
rho_trial = TrialFunction(Vr)
lhs = gamma * rho_trial * dx
rhs = gamma * (rho_b * theta_b / theta0) * dx
rho_problem = LinearVariationalProblem(lhs, rhs, rho0)
rho_solver = LinearVariationalSolver(rho_problem)
rho_solver.solve()

state.set_reference_profiles([('rho', rho_b),
                              ('theta', theta_b)])

Set up the transport schemes

In [16]:
VDG1 = state.spaces("DG1", "DG", 1)
VCG1 = FunctionSpace(mesh, "CG", 1)
Vt_brok = FunctionSpace(mesh, BrokenElement(Vt.ufl_element()))
Vu_DG1 = VectorFunctionSpace(mesh, VDG1.ufl_element())
Vu_CG1 = VectorFunctionSpace(mesh, "CG", 1)
Vu_brok = FunctionSpace(mesh, BrokenElement(Vu.ufl_element()))

u_opts = RecoveredOptions(embedding_space=Vu_DG1,
                          recovered_space=Vu_CG1,
                          broken_space=Vu_brok,
                          boundary_method=Boundary_Method.dynamics)
rho_opts = RecoveredOptions(embedding_space=VDG1,
                            recovered_space=VCG1,
                            broken_space=Vr,
                            boundary_method=Boundary_Method.dynamics)
theta_opts = RecoveredOptions(embedding_space=VDG1,
                              recovered_space=VCG1,
                              broken_space=Vt_brok)

Set up the time-stepping choices for the transported fields. We will use SSPRK3 for all prognostic variables.

In [17]:
limiter = None

transported_fields = [SSPRK3(state, "u", options=u_opts),
                      SSPRK3(state, "rho", options=rho_opts),
                      SSPRK3(state, "theta", options=theta_opts, limiter=limiter)]

Set up the linear solver and the time-stepper. We don't need to have any diffusion schemes.

In [19]:
# Set up linear solver
linear_solver = CompressibleSolver(state, eqns)

diffusion_schemes = []

# build time stepper
stepper = CrankNicolson(state, eqns, transported_fields,
                        linear_solver=linear_solver,
                        diffusion_schemes=diffusion_schemes)

Run!

In [20]:
stepper.run(t=0, tmax=tmax)

/home/timandrews/firedrake/src/firedrake/firedrake/checkpointing.py:89: DeprecationWarning: DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.
  warnings.warn("DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.",
gusto:INFO at start of timestep, t=0, dt=1.0
tsfc:WARNING Estimated quadrature degree (11, 11) more than tenfold greater than any argument/coefficient degree (max (1, 1))
tsfc:WARNING Estimated quadrature degree (12, 12) more than tenfold greater than any argument/coefficient degree (max (1, 1))
tsfc:WARNING Estimated quadrature degree (11, 11) more than tenfold greater than any argument/coefficient degree (max (1, 1))
tsfc:WARNING Estimated quadrature degree (11, 12) more than tenfold greater than any argument/coefficient degree (max (1, 1))
gusto:INFO at start of timestep, t=1.0, dt=1.0
gusto:INFO at start of timestep, t=2.0, dt=1.0
gusto:INFO at start of timestep, t=3.0, dt=1.0
gusto:INFO at s

gusto:INFO at start of timestep, t=154.0, dt=1.0
gusto:INFO at start of timestep, t=155.0, dt=1.0
gusto:INFO at start of timestep, t=156.0, dt=1.0
gusto:INFO at start of timestep, t=157.0, dt=1.0
gusto:INFO at start of timestep, t=158.0, dt=1.0
gusto:INFO at start of timestep, t=159.0, dt=1.0
gusto:INFO at start of timestep, t=160.0, dt=1.0
gusto:INFO at start of timestep, t=161.0, dt=1.0
gusto:INFO at start of timestep, t=162.0, dt=1.0
gusto:INFO at start of timestep, t=163.0, dt=1.0
gusto:INFO at start of timestep, t=164.0, dt=1.0
gusto:INFO at start of timestep, t=165.0, dt=1.0
gusto:INFO at start of timestep, t=166.0, dt=1.0
gusto:INFO at start of timestep, t=167.0, dt=1.0
gusto:INFO at start of timestep, t=168.0, dt=1.0
gusto:INFO at start of timestep, t=169.0, dt=1.0
gusto:INFO at start of timestep, t=170.0, dt=1.0
gusto:INFO at start of timestep, t=171.0, dt=1.0
gusto:INFO at start of timestep, t=172.0, dt=1.0
gusto:INFO at start of timestep, t=173.0, dt=1.0
gusto:INFO at start 

gusto:INFO at start of timestep, t=322.0, dt=1.0
gusto:INFO at start of timestep, t=323.0, dt=1.0
gusto:INFO at start of timestep, t=324.0, dt=1.0
gusto:INFO at start of timestep, t=325.0, dt=1.0
gusto:INFO at start of timestep, t=326.0, dt=1.0
gusto:INFO at start of timestep, t=327.0, dt=1.0
gusto:INFO at start of timestep, t=328.0, dt=1.0
gusto:INFO at start of timestep, t=329.0, dt=1.0
gusto:INFO at start of timestep, t=330.0, dt=1.0
gusto:INFO at start of timestep, t=331.0, dt=1.0
gusto:INFO at start of timestep, t=332.0, dt=1.0
gusto:INFO at start of timestep, t=333.0, dt=1.0
gusto:INFO at start of timestep, t=334.0, dt=1.0
gusto:INFO at start of timestep, t=335.0, dt=1.0
gusto:INFO at start of timestep, t=336.0, dt=1.0
gusto:INFO at start of timestep, t=337.0, dt=1.0
gusto:INFO at start of timestep, t=338.0, dt=1.0
gusto:INFO at start of timestep, t=339.0, dt=1.0
gusto:INFO at start of timestep, t=340.0, dt=1.0
gusto:INFO at start of timestep, t=341.0, dt=1.0
gusto:INFO at start 

gusto:INFO at start of timestep, t=490.0, dt=1.0
gusto:INFO at start of timestep, t=491.0, dt=1.0
gusto:INFO at start of timestep, t=492.0, dt=1.0
gusto:INFO at start of timestep, t=493.0, dt=1.0
gusto:INFO at start of timestep, t=494.0, dt=1.0
gusto:INFO at start of timestep, t=495.0, dt=1.0
gusto:INFO at start of timestep, t=496.0, dt=1.0
gusto:INFO at start of timestep, t=497.0, dt=1.0
gusto:INFO at start of timestep, t=498.0, dt=1.0
gusto:INFO at start of timestep, t=499.0, dt=1.0
gusto:INFO at start of timestep, t=500.0, dt=1.0
gusto:INFO at start of timestep, t=501.0, dt=1.0
gusto:INFO at start of timestep, t=502.0, dt=1.0
gusto:INFO at start of timestep, t=503.0, dt=1.0
gusto:INFO at start of timestep, t=504.0, dt=1.0
gusto:INFO at start of timestep, t=505.0, dt=1.0
gusto:INFO at start of timestep, t=506.0, dt=1.0
gusto:INFO at start of timestep, t=507.0, dt=1.0
gusto:INFO at start of timestep, t=508.0, dt=1.0
gusto:INFO at start of timestep, t=509.0, dt=1.0
gusto:INFO at start 

gusto:INFO at start of timestep, t=658.0, dt=1.0
gusto:INFO at start of timestep, t=659.0, dt=1.0
gusto:INFO at start of timestep, t=660.0, dt=1.0
gusto:INFO at start of timestep, t=661.0, dt=1.0
gusto:INFO at start of timestep, t=662.0, dt=1.0
gusto:INFO at start of timestep, t=663.0, dt=1.0
gusto:INFO at start of timestep, t=664.0, dt=1.0
gusto:INFO at start of timestep, t=665.0, dt=1.0
gusto:INFO at start of timestep, t=666.0, dt=1.0
gusto:INFO at start of timestep, t=667.0, dt=1.0
gusto:INFO at start of timestep, t=668.0, dt=1.0
gusto:INFO at start of timestep, t=669.0, dt=1.0
gusto:INFO at start of timestep, t=670.0, dt=1.0
gusto:INFO at start of timestep, t=671.0, dt=1.0
gusto:INFO at start of timestep, t=672.0, dt=1.0
gusto:INFO at start of timestep, t=673.0, dt=1.0
gusto:INFO at start of timestep, t=674.0, dt=1.0
gusto:INFO at start of timestep, t=675.0, dt=1.0
gusto:INFO at start of timestep, t=676.0, dt=1.0
gusto:INFO at start of timestep, t=677.0, dt=1.0
gusto:INFO at start 

gusto:INFO at start of timestep, t=826.0, dt=1.0
gusto:INFO at start of timestep, t=827.0, dt=1.0
gusto:INFO at start of timestep, t=828.0, dt=1.0
gusto:INFO at start of timestep, t=829.0, dt=1.0
gusto:INFO at start of timestep, t=830.0, dt=1.0
gusto:INFO at start of timestep, t=831.0, dt=1.0
gusto:INFO at start of timestep, t=832.0, dt=1.0
gusto:INFO at start of timestep, t=833.0, dt=1.0
gusto:INFO at start of timestep, t=834.0, dt=1.0
gusto:INFO at start of timestep, t=835.0, dt=1.0
gusto:INFO at start of timestep, t=836.0, dt=1.0
gusto:INFO at start of timestep, t=837.0, dt=1.0
gusto:INFO at start of timestep, t=838.0, dt=1.0
gusto:INFO at start of timestep, t=839.0, dt=1.0
gusto:INFO at start of timestep, t=840.0, dt=1.0
gusto:INFO at start of timestep, t=841.0, dt=1.0
gusto:INFO at start of timestep, t=842.0, dt=1.0
gusto:INFO at start of timestep, t=843.0, dt=1.0
gusto:INFO at start of timestep, t=844.0, dt=1.0
gusto:INFO at start of timestep, t=845.0, dt=1.0
gusto:INFO at start 

gusto:INFO at start of timestep, t=994.0, dt=1.0
gusto:INFO at start of timestep, t=995.0, dt=1.0
gusto:INFO at start of timestep, t=996.0, dt=1.0
gusto:INFO at start of timestep, t=997.0, dt=1.0
gusto:INFO at start of timestep, t=998.0, dt=1.0
gusto:INFO at start of timestep, t=999.0, dt=1.0
gusto:INFO TIMELOOP complete. t=1000.0, tmax=1000.0
